#                           Image Stitching and Preprocessing Script

Given was a URL with slices of MRI images, which appeared to be taken on different magnification levels, spread through different subdirectories in a folder.

The task was to collect these images and stitch them to recreate the original MRI images from the slices given. Another task was to then preprocess and prepare these images to be used in a neural network for training.

### Importing Required Libraries

In [ ]:
"""
Created on Wed Aug 14 17:06:50 2019

@author: Raghav Avasthi

The script helps stitching images together and preprocess them for prepareing the 
stitched images to be fed into a neural network. 
"""

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
import skimage as ski


## Stitching MRI Image Segments

### It converts segments of images such as this 

![title](images/segments1.png)

### to this ..

![title](images/full.jpg)

In [ ]:
def stitch_segments(folder_path, magnification_level, input_check_exec = True):
    '''
    Description
    -----------
    Given the path of the folder containing segmented images and a specified magnification level, the function stitches all the image segments to create a complete bigger image.
    It expects the image segments to be named in the format z-x-y.jpg where 'z' corresponds to the magnification level at which that iamge was taken and 'x' and 'y' correspond
    to the segmentation indexes according to which segments were made in the horizontal(latitudnal) and vertical(longitudinal) directions respectively.
    
    Parameters
    ----------
        folder_path:
            Path of the folder containing segments of an MRI image, on different magnification levels.
            
        magnification_level:
            Integer depicting the desired magnification level for which stitching needs to be done. 
        
        input_check_exec:
            Accepts a boolean value. If True, code will stop execution if any od the checks made on input fails. Else, the function would return 
            'None' so that execution of the code can be continued. Default value of this variable is TRUE.
        
    Returns
    -------
        lat_concat:
            Returns a 3 channel 'uint8' numpy matrix stitched image made up of all the segments in a given magnification level.
            Returns None if inputs to the function do not pass the initial checks
    '''
    
    

### Placing checks for incoming values in the function

In [ ]:
    if input_check_exec:
        if not os.path.exists(folder_path):
            print('\n' + 'ERROR: Folder path does not exist. Stopping code execution' + '\n')
            sys.exit()
            
        list_crops = os.listdir(folder_path) # listing the names of all image segment from folder_mri, in ascending order
        if not list_crops:
            print('\n' + 'ERROR: Folder path does not contain any files. Stopping code execution' + '\n')
            sys.exit()
        
        if not isinstance(magnification_level, int):
            print('\n' + 'ERROR: Magnification level supplied is not an integer. Stopping code execution' + '\n')
            sys.exit()
    
    else:
        if not os.path.exists(folder_path):
            print('\n' + 'ERROR: Folder path does not exist. Function is stopping execution and is returning "None". ' + '\n')
            return None
            
        list_crops = os.listdir(folder_path) # listing the names of all image segment from folder_mri, in ascending order
        if not list_crops:
            print('\n' + 'ERROR: Folder path does not contain any files. Function is stopping execution and is returning "None".' + '\n')
            return None
        
        if not isinstance(magnification_level, int):
            print('\n' + 'ERROR: Magnification level supplied is not an integer. Function is stopping execution and is returning "None".' + '\n')
            return None   
            
            
    ''' ALL INPUT CHECKS ARE COMPLETED FOR THE FUNCTION TO CONTINUE EXECUTION '''        

### Looping through all segments to stitch the image. For each image, memory requirments increase linearly and drops to 0 suddenly as image changes

In [ ]:
    mag = 0  # Initialization of the iterating mag variable. Mag refers to the magnification specified in the image name being parsed
    img_index = 1 # parsing index  
    lat_index = 0 # index parsing over latitudenal values 
    long_index = 0 # index parsing over longitudinal values
    
    while mag <= magnification_level: # while loop parses till it goes beyond the required magnification
        mag, lat, long  = (list_crops[img_index].split('.')[0]).split('-') 
        # disintigrating the magnification level, latitude, and the longitude of an image segment from its image name
        mag = int(mag)
        lat = int(lat)
        long = int(long)
        
        if mag == magnification_level:
    # ===============================================================================================================================================
    # The while loop here has nested if else statements for parsing and stitching each image. After checking if the magnitude, stitching is done
    # longitudnally first, keeping the latitude(horizontal axis) constant. Once a longitudinal segment of the image is stitched, it is latitudnally 
    # (horizonally) concatinated with other longnitudianl segments. This is an iterative process
    # ===============================================================================================================================================
            if lat == lat_index: 
    # checks if the latitude is changing while parsing images, if it does not, it does longitudinal concatination, else sets a starting pont for it
                image = np.asarray(cv2.imread(os.path.join(folder_path,list_crops[img_index])))
                if long == long_index:
                    concat_img = image  # Setting the starting point for longitudinal concatination
                elif long == long_index + 1:
                    concat_img = np.concatenate((concat_img, image), axis = 0) # Longitudinal concatination takes place here
                    long_index  = long_index + 1
                img_index = img_index + 1
            else:
                long_index = 0 
                old_concat = concat_img # saving the completed longitudinal concatination in a temporary variable, for use in latitudinal concatination later
                image = np.asarray(cv2.imread(os.path.join(folder_path,list_crops[img_index])))
                concat_img = image # Setting the starting point for longitudinal concatination when latitude changes
                lat_index = lat_index + 1
                if lat_index <=1:
                    lat_concat = old_concat
                else:
                    lat_concat = np.concatenate((lat_concat, old_concat), axis = 1) # Concatinating completed longitudinal segments, in horizontal direction
        else:
            img_index = img_index + 1
    lat_concat = np.concatenate((lat_concat, concat_img), axis = 1) 
    lat_concat = cv2.cvtColor(lat_concat, cv2.COLOR_RGB2BGR)
    return lat_concat      

## Basic Preprocessing to Crop and Resize the Stitched Image

Stitched images many times have very less usefull data in them. Taking the example of the image given above, one can notice that the MRI image resides only in approximately 40% of the image. Since the images are too big to be fed into a neural network, we can crop them to emphasize on the useful data in the image.

Images are then padded with their border pixels to make them square in shape and then they are resized to the desired level. In this example all incoming images will be resized to 224 x 224 x 3 size as they are to be fed into a MobileNet V2 for classification and network training.

### This function turns images from this 

![title](images/full.jpg)

### to this .. 

![title](images/full_cropped.png)

In [ ]:
def crop_n_resize(image, border_margin = 10, size_x = 512, size_y = 512):
    '''
    Description
    -----------
    Given an image in 'uint8' format, the function does the following operations on the image to standardize it.
    > Finds the largest bounding box in the image containing the whole MRI object in the image.
    > Crops the image as per the bounding box leaving a margin specified by 'border_margin'
    > Pads the image to make it square shape. Padding is done by replicating the border pixels
    > Resizes the image to the desired size
    
    Parameters
    ----------
        image:
            Accepts a 3-channel image in uint8 format.
            
        border_margin:
            Accepts an integer depicting the amount of margin to be kept while cropping the image from its smallest bounding box possible. 
            Default is 10.
        
        size_x:
            Accepts an interger value. It is the desired width of the resultant image after resize. Default is 512.
        
        size_y:
            Accepts an interger value. It is the desired height of the resultant image after resize. Default is 512.
        
    Returns
    -------
        lat_concat:
            Returns a 3 channel 'uint8' numpy matrix stitched image made up of all the segments in a given magnification level.
            Returns None if inputs to the function do not pass the initial checks.
    '''

### Placing checks for incoming values in the function

In [ ]:
    if not isinstance(border_margin, int):
        print('\n' + 'ERROR: Border Margin supplied is not an integer. Stopping code execution' + '\n')
        sys.exit()
    if not isinstance(size_x, int):
        print('\n' + 'ERROR: Desired width of the output image supplied is not an integer. Stopping code execution' + '\n')
        sys.exit()
    if not isinstance(size_y, int):
        print('\n' + 'ERROR: Desired height of the output image supplied is not an integer. Stopping code execution' + '\n')
        sys.exit()
   

### Cropping, Padding and Resizing the Images 

In [ ]:
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # converting the image into grayscale
    _,bw = cv2.threshold(gray,210,255,cv2.THRESH_BINARY_INV) # converting graysacle image into binary for morphological operations
    bw = cv2.erode(bw, np.ones((5,5), np.uint8), 10) # Erosion and dilution to remove any salt and pepper noise
    bw = cv2.dilate(bw, np.ones((5,5), np.uint8), 10)
    label_bw = ski.measure.label(bw)
    del bw
    box_concat = None
    for region in ski.measure.regionprops(label_bw):
        box = np.asarray(region.bbox).reshape(4,1) # Calcualting bounding box for all objects in the image
        if box_concat is None: box_concat = box
        box_concat = np.concatenate((box_concat, box), axis = 1) # Concatinating all bounding box results to find the extrema points
    del label_bw
    (gray_row, gray_col) = gray.shape
    row_min = max(0, min(box_concat[0]) - border_margin)
    col_min = max(0, min(box_concat[1]) - border_margin) # Calcualted the extrema points
    row_max = min(gray_row, max(box_concat[2]) + border_margin)
    col_max = min(gray_col, max(box_concat[3]) + border_margin)
    del box_concat
    
    crop_im = image[row_min:row_max, col_min:col_max,:] # Cropped the image as per calculated extremas
    (r,c,_) = crop_im.shape
    
    if r > c: # Made the image in square shape 
        diff = r-c
        pad = int(diff / 2)
        crop_im = cv2.copyMakeBorder(crop_im,0,0,pad,pad,cv2.BORDER_REPLICATE)# Used padding by replicating the border pixels to make square 
    elif r < c:
        diff = c-r
        pad = int(diff / 2)
        crop_im = cv2.copyMakeBorder(crop_im,pad,pad,0,0,cv2.BORDER_REPLICATE)
    crop_im = cv2.resize(crop_im,(size_x,size_y)) # resized the resultant image to the deisred size
    return crop_im

## Main Function to execute all preprocessing steps in a single go! 

In [ ]:
if __name__ == '__main__':
    data_dir = r'C:\Users\Raghav Avasthi\Desktop\mouse brain\Brain_Dataset_Color\validate\horizontal'
    save_dir = r'C:\Users\Raghav Avasthi\Desktop\mouse brain\Brain_Dataset_Color_Pre\validate\horizontal'
    list_mri = os.listdir(data_path)
    index=0
    for image_name in list_mri:
        image = np.asarray(cv2.imread(os.path.join(data_dir,image_name)))
        folder_mri = os.path.join(data_dir, list_mri[index]) # selecting a single folder which corresponds to a single MRI image
        
        image = stitch_segments(folder_mri, 3) # Applying functon to stitch MRI image segments 
        
        # Applying function to select the best portion of image data from the MRI image, pad it and resize it into desired shape.
        final_im = crop_n_resize(image, size_x = 224, size_y = 224)
        
        save_path = os.path.join(save_dir, image_name) # creating a path where final image will be saved, complete with the name of the image.

        cv2.imwrite(save_path, final_im) # Saving the image in the directory specified in 'save_dir' with preserving the name of the MRI image.
